In [ ]:
# 0. Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 1. Import Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.metrics import classification_report, roc_auc_score
from scipy.stats.mstats import winsorize
from imblearn.over_sampling import SMOTE

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, MultiHeadAttention, LayerNormalization, Lambda
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical

# 2. Set Seed untuk Konsistensi
SEED = 44
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)
os.environ['TF_DETERMINISTIC_OPS'] = '1'

# 3. Load Dataset
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Skripsi/Dataset/diabetes_012_health_indicators_BRFSS2015.csv')
X = data.drop('Diabetes_012', axis=1)
y = data['Diabetes_012']

# 4. Preprocessing
X['BMI'] = winsorize(X['BMI'], limits=[0.005, 0.005])
X['MentHlth'] = np.where(X['MentHlth'] > 30, 30, X['MentHlth'])
X['PhysHlth'] = np.where(X['PhysHlth'] > 30, 30, X['PhysHlth'])

robust_features = ['BMI', 'MentHlth', 'PhysHlth']
minmax_features = ['Age', 'Education', 'Income', 'GenHlth']

scaler_robust = RobustScaler()
scaler_minmax = MinMaxScaler()

X[robust_features] = scaler_robust.fit_transform(X[robust_features])
X[minmax_features] = scaler_minmax.fit_transform(X[minmax_features])

# 5. Split Data (Train/Test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=SEED, stratify=y
)

# 6. SMOTE on Train Data Only
smote = SMOTE(random_state=SEED)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Simpan hasil SMOTE jika dibutuhkan
# df_train_smote = pd.DataFrame(X_train_smote, columns=X_train.columns)
# df_train_smote['Diabetes_012'] = y_train_smote
# df_train_smote.to_csv('/content/drive/MyDrive/Colab Notebooks/Skripsi/Dataset/Output DNN+SMOTE.csv', index=False)

# 7. One-hot Encoding
y_train_encoded = to_categorical(y_train_smote)
y_test_encoded = to_categorical(y_test)

# 8. Split Train → Train Main & Validation
X_train_main, X_val, y_train_main, y_val = train_test_split(
    X_train_smote, y_train_encoded, test_size=0.2, random_state=SEED, stratify=y_train_smote
)

# 9. Build DNN + Self-Attention Model
input_layer = Input(shape=(X.shape[1],))
x = Dense(64, activation='relu')(input_layer)
x = Dense(32, activation='relu')(x)
x = Dense(16, activation='relu')(x)

# Expand dimension for self-attention
x_expanded = Lambda(lambda t: tf.expand_dims(t, axis=1))(x)

# Self-Attention Layer
attn_output = MultiHeadAttention(num_heads=4, key_dim=4)(x_expanded, x_expanded)
attn_output = LayerNormalization()(attn_output + x_expanded)

# Flatten back to 2D
attn_output_squeezed = Lambda(lambda t: tf.squeeze(t, axis=1))(attn_output)

# Output Layer
output_layer = Dense(3, activation='softmax')(attn_output_squeezed)

# Final Model
model = Model(inputs=input_layer, outputs=output_layer)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# 10. Train Model
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(
    X_train_main, y_train_main,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=50,
    callbacks=[early_stop],
    verbose=1
)

# 11. Evaluate on Test Set
y_pred_prob = model.predict(X_test)
y_pred_class = np.argmax(y_pred_prob, axis=1)
y_true_class = np.argmax(y_test_encoded, axis=1)

# 12. Metrik Evaluasi
print("\n=== Classification Report (Macro) Model DNN + SMOTE + SA ===")
print(classification_report(y_true_class, y_pred_class, digits=4))
roc_auc = roc_auc_score(y_test_encoded, y_pred_prob, average='macro', multi_class='ovr')
print(f"\nMacro-average ROC AUC: {roc_auc:.4f}")


Mounted at /content/drive
Epoch 1/100
8207/8207 ━━━━━━━━━━━━━━━━━━━━ 45s 5ms/step - accuracy: 0.5272 - loss: 0.9409 - val_accuracy: 0.5793 - val_loss: 0.8799
Epoch 2/100
8207/8207 ━━━━━━━━━━━━━━━━━━━━ 81s 5ms/step - accuracy: 0.5867 - loss: 0.8739 - val_accuracy: 0.6030 - val_loss: 0.8491
Epoch 3/100
8207/8207 ━━━━━━━━━━━━━━━━━━━━ 40s 5ms/step - accuracy: 0.6148 - loss: 0.8312 - val_accuracy: 0.6191 - val_loss: 0.8265
Epoch 4/100
8207/8207 ━━━━━━━━━━━━━━━━━━━━ 40s 5ms/step - accuracy: 0.6315 - loss: 0.8041 - val_accuracy: 0.6305 - val_loss: 0.8108
Epoch 5/100
8207/8207 ━━━━━━━━━━━━━━━━━━━━ 45s 5ms/step - accuracy: 0.6431 - loss: 0.7855 - val_accuracy: 0.6376 - val_loss: 0.7975
Epoch 6/100
8207/8207 ━━━━━━━━━━━━━━━━━━━━ 77s 5ms/step - accuracy: 0.6509 - loss: 0.7724 - val_accuracy: 0.6472 - val_loss: 0.7820
Epoch 7/100
8207/8207 ━━━━━━━━━━━━━━━━━━━━ 42s 5ms/step - accuracy: 0.6574 - loss: 0.7623 - val_accuracy: 0.6508 - val_loss: 0.7746
Epoch 8/100
8207/8207 ━━━━━━━━━━━━━━━━━━━━ 44s 5ms